In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchbnn as bnn  # torchbnn library for BNN layers
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [2]:
class BayesianNN(nn.Module):
    def __init__(self, input_dim):
        super(BayesianNN, self).__init__()
        # Define prior parameters
        prior_mu = 0.0  # Mean of the prior distribution
        prior_sigma = 0.1  # Standard deviation of the prior distribution
        
        # Initialize Bayesian layers with the specified priors
        self.fc1 = bnn.BayesLinear(prior_mu, prior_sigma, in_features=input_dim, out_features=64)
        self.fc2 = bnn.BayesLinear(prior_mu, prior_sigma, in_features=64, out_features=32)
        self.fc3 = bnn.BayesLinear(prior_mu, prior_sigma, in_features=32, out_features=1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [3]:
# Training function 
def train_bayesian_nn(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred.squeeze(), y_batch)  # Squeeze to match dimensions
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [4]:
# Evaluation function
def evaluate_bayesian_nn(model, loader):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for X_batch, _ in loader:
            y_pred = model(X_batch)
            all_preds.extend(y_pred.round().squeeze().cpu().numpy())
    return np.array(all_preds)

In [5]:
# Function to run BNN for each value of n
def run_bayesian_nn(n, hyper_params):
    # Load data
    X = np.load(f'Datasets/kryptonite-{n}-X.npy')
    y = np.load(f'Datasets/kryptonite-{n}-y.npy')
    
    # Split data into training, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    # Convert data to PyTorch tensors and create DataLoaders
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
    X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)
    
    # Convert data to PyTorch tensors and create DataLoaders
    train_loader = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                                torch.tensor(y_train, dtype=torch.float32)),
                                  batch_size=hyper_params['batch_size'], shuffle=True)
    val_loader = DataLoader(TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                                              torch.tensor(y_val, dtype=torch.float32)),
                                batch_size=hyper_params['batch_size'])
    # Test the best model on the test set
    test_loader = DataLoader(TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                           torch.tensor(y_test, dtype=torch.float32)),
                             batch_size=32)
    # Initialize the model, loss function, and optimizer
    input_dim = X_train.shape[1]
    model = BayesianNN(input_dim=input_dim)
    model.fc1 = bnn.BayesLinear(hyper_params['prior_mu'], hyper_params['prior_sigma'], 
                                in_features=input_dim, out_features=hyper_params['layer1_units'])
    model.fc2 = bnn.BayesLinear(hyper_params['prior_mu'], hyper_params['prior_sigma'], 
                                in_features=hyper_params['layer1_units'], out_features=hyper_params['layer2_units'])
    model.fc3 = bnn.BayesLinear(hyper_params['prior_mu'], hyper_params['prior_sigma'], 
                                in_features=hyper_params['layer2_units'], out_features=1)
    
    criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
    optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params['learning_rate'])
    
    # Train the model and track training loss
    training_losses = []
    for epoch in range(hyper_params['epochs']):
        train_loss = train_bayesian_nn(model, train_loader, criterion, optimizer)
        training_losses.append(train_loss)
        print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}")
    
    # Validate the model
    y_val_pred = evaluate_bayesian_nn(model, val_loader)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    print(f"Validation Accuracy for n={n}: {val_accuracy:.4f}")
    
    # Test the model
    y_test_pred = evaluate_bayesian_nn(model, test_loader)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Test Accuracy for n={n}: {test_accuracy:.4f}")
    
    return test_accuracy

In [ ]:
# Define the hyperparameter tuning function
def tune_hyperparameters(n, param_grid):
    # Load data
    X = np.load(f'Datasets/kryptonite-{n}-X.npy')
    y = np.load(f'Datasets/kryptonite-{n}-y.npy')
    
    # Split data into training, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    best_accuracy = 0
    best_params = None
    
    for params in ParameterGrid(param_grid):
        print(f"Tuning with params: {params}")
        input_dim = X_train.shape[1]
        # Set up model with current hyperparameters
        model = BayesianNN(input_dim=input_dim)
        model.fc1 = bnn.BayesLinear(params['prior_mu'], params['prior_sigma'], 
                                    in_features=input_dim, out_features=params['layer1_units'])
        model.fc2 = bnn.BayesLinear(params['prior_mu'], params['prior_sigma'], 
                                    in_features=params['layer1_units'], out_features=params['layer2_units'])
        model.fc3 = bnn.BayesLinear(params['prior_mu'], params['prior_sigma'], 
                                    in_features=params['layer2_units'], out_features=1)
        
        # Convert data to PyTorch tensors and create DataLoaders
        train_loader = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                                torch.tensor(y_train, dtype=torch.float32)),
                                  batch_size=params['batch_size'], shuffle=True)
        val_loader = DataLoader(TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                                              torch.tensor(y_val, dtype=torch.float32)),
                                batch_size=params['batch_size'])
        
        # Define the loss function and optimizer
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
        
        # Train the model and track training loss
        training_losses = []
        for epoch in range(params['epochs']):
            train_loss = train_bayesian_nn(model, train_loader, criterion, optimizer)
            training_losses.append(train_loss)
            print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}")
        
        # Validate the model
        y_val_pred = evaluate_bayesian_nn(model, val_loader)
        val_accuracy = accuracy_score(y_val, y_val_pred)
        print(f"Validation Accuracy: {val_accuracy:.4f}")
        
        # Track the best hyperparameters based on validation accuracy
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_params = params
            best_model = model
    
    # Test the best model on the test set
    test_loader = DataLoader(TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                           torch.tensor(y_test, dtype=torch.float32)),
                             batch_size=32)
    y_test_pred = evaluate_bayesian_nn(best_model, test_loader)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print("Best Hyperparameters:", best_params)
    print("Best Validation Accuracy:", best_accuracy)
    print("Test Accuracy:", test_accuracy)
    
    return best_params, best_accuracy, test_accuracy, training_losses


# Define the hyperparameter grid
param_grid = {
    'prior_mu': [0.0, 0.1, 0.2],  # Mean of prior distribution
    'prior_sigma': [0.1, 0.2, 0.3],  # Standard deviation of prior
    'layer1_units': [32, 64, 128],  # Units in the first layer
    'layer2_units': [16, 32, 64],  # Units in the second layer
    'batch_size': [16, 32, 64],  # Batch sizes
    'learning_rate': [0.001, 0.005, 0.01],  # Learning rates
    'epochs': [10, 20, 30]  # Number of training epochs
}


# Run hyperparameter tuning for the Bayesian Neural Network on dataset n=9
best_params, best_accuracy, test_accuracy, training_losses = tune_hyperparameters(9, param_grid)

Tuning with params: {'batch_size': 16, 'epochs': 10, 'layer1_units': 32, 'layer2_units': 16, 'learning_rate': 0.001, 'prior_mu': 0.0, 'prior_sigma': 0.1}
Epoch 1, Train Loss: 0.6967
Epoch 2, Train Loss: 0.6945
Epoch 3, Train Loss: 0.6951
Epoch 4, Train Loss: 0.6936
Epoch 5, Train Loss: 0.6945
Epoch 6, Train Loss: 0.6940
Epoch 7, Train Loss: 0.6930
Epoch 8, Train Loss: 0.6945
Epoch 9, Train Loss: 0.6934
Epoch 10, Train Loss: 0.6936
Validation Accuracy: 0.5050
Tuning with params: {'batch_size': 16, 'epochs': 10, 'layer1_units': 32, 'layer2_units': 16, 'learning_rate': 0.001, 'prior_mu': 0.0, 'prior_sigma': 0.2}
Epoch 1, Train Loss: 0.7084
Epoch 2, Train Loss: 0.7040
Epoch 3, Train Loss: 0.6991
Epoch 4, Train Loss: 0.6968
Epoch 5, Train Loss: 0.6974
Epoch 6, Train Loss: 0.6955
Epoch 7, Train Loss: 0.6955
Epoch 8, Train Loss: 0.6976
Epoch 9, Train Loss: 0.6939
Epoch 10, Train Loss: 0.6941
Validation Accuracy: 0.5120
Tuning with params: {'batch_size': 16, 'epochs': 10, 'layer1_units': 32, '

KeyboardInterrupt: 

In [15]:
# Run for each n value and collect accuracies
results = []

hyper_params = {
    'prior_mu': 0.0,
    'prior_sigma': 0.1,
    'layer1_units': 64,
    'layer2_units': 32,
    'batch_size': 32,
    'learning_rate': 0.01,
    'epochs': 20
}
possible_n_vals = [18]
for n in tqdm(possible_n_vals):
    accuracy = run_bayesian_nn(n, hyper_params)
    results.append((n, accuracy))

print("Accuracies across different n values:", results)

# Threshold grid for each n value
thresh_grid = {
    '9': 0.95,  
    '12': 0.925,
    '15': 0.90,
    '18': 0.875,
    '24': 0.80,
    '30': 0.75,
    '45': 0.70
}


  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Martin Nguyen\AppData\Local\Temp\ipykernel_11064\1454362721.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_loader = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float32),
C:\Users\Martin Nguyen\AppData\Local\Temp\ipykernel_11064\1454362721.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_train, dtype=torch.float32)),
C:\Users\Martin Nguyen\AppData\Local\Temp\ipykernel_11064\1454362721.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_loader =

Epoch 1, Train Loss: 0.6962
Epoch 2, Train Loss: 0.6942
Epoch 3, Train Loss: 0.6941
Epoch 4, Train Loss: 0.6937
Epoch 5, Train Loss: 0.6936
Epoch 6, Train Loss: 0.6933
Epoch 7, Train Loss: 0.6935
Epoch 8, Train Loss: 0.6936
Epoch 9, Train Loss: 0.6935
Epoch 10, Train Loss: 0.6933
Epoch 11, Train Loss: 0.6935
Epoch 12, Train Loss: 0.6933
Epoch 13, Train Loss: 0.6935
Epoch 14, Train Loss: 0.6934
Epoch 15, Train Loss: 0.6934
Epoch 16, Train Loss: 0.6935
Epoch 17, Train Loss: 0.6932
Epoch 18, Train Loss: 0.6935
Epoch 19, Train Loss: 0.6937
Epoch 20, Train Loss: 0.6935
Validation Accuracy for n=18: 0.4989


100%|██████████| 1/1 [00:26<00:00, 26.10s/it]

Test Accuracy for n=18: 0.4957
Accuracies across different n values: [(18, 0.49574074074074076)]
